In [31]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


In [32]:
main_url = "http://books.toscrape.com/index.html"
result = requests.get(main_url)

result.text[:1000]

'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n        <meta name="created" content="24th Jun 2016 09:29" />\n        <meta name="description" content="" />\n        <meta name="viewport" content="width=device-width" />\n        <meta name="robots" content="NOARCHIVE,NOCACHE" />\n\n        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->\n        <!--[if lt IE 9]>\n        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n        <![endif]-->\n\n        \n            <link rel="shortcut icon" href

In [33]:
soup = BeautifulSoup(result.text, 'html.parser')

print(soup.prettify()[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:29" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="static/oscar/css/styles.css" rel="stylesheet" type="tex

In [34]:
def getAndParseURL(url):
    result = requests.get(url)
    soup = BeautifulSoup(result.text, 'html.parser')
    return(soup)

In [35]:
# soup.find("article", class_ = "product_pod")

soup.find("article", class_= "product_pod").div.a

soup.find("article", class_= "product_pod").div.a.get('href')



'catalogue/a-light-in-the-attic_1000/index.html'

In [36]:
main_page_products_urls = [x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")]

print(str(len(main_page_products_urls)) + " fetched products URLs")
print("One Example:")
print(main_page_products_urls[:5])

20 fetched products URLs
One Example:
['catalogue/a-light-in-the-attic_1000/index.html', 'catalogue/tipping-the-velvet_999/index.html', 'catalogue/soumission_998/index.html', 'catalogue/sharp-objects_997/index.html', 'catalogue/sapiens-a-brief-history-of-humankind_996/index.html']


In [47]:
def getBooksURLs(url):
    soup = getAndParseURL(url)
    return(["/".join(url.split("/")[:-1]) + "/" + x.div.a.get('href') for x in soup.findAll("article", class_ = "product_pod")])


In [39]:
import re
pages_urls = [main_url]

soup = getAndParseURL(pages_urls[0])
# print(soup.prettify()[:1000])



while len(soup.findAll("a", href=re.compile("page"))) == 2 or len(pages_urls) == 1:
    new_url = "/".join(pages_urls[-1].split("/")[:-1]) + "/" + soup.findAll("a", href=re.compile("page"))[-1].get("href")
    
    pages_urls.append(new_url)
    
    soup = getAndParseURL(new_url)
    
    
print(str(len(pages_urls)) + " fetched URLs")
print("some examples:")
print(pages_urls[:5])

50 fetched URLs
some examples:
['http://books.toscrape.com/index.html', 'http://books.toscrape.com/catalogue/page-2.html', 'http://books.toscrape.com/catalogue/page-3.html', 'http://books.toscrape.com/catalogue/page-4.html', 'http://books.toscrape.com/catalogue/page-5.html']


In [40]:
result = requests.get("http://books.toscrape.com/catalogue/page-50.html")
print("status code for page 50: " + str(result.status_code))
# no erro page


result = requests.get("http://books.toscrape.com/catalogue/page-51.html")
print("status code for page 50: " + str(result.status_code))
# 404 error page

status code for page 50: 200
status code for page 50: 404


In [41]:
pages_urls = []

new_page = "http://books.toscrape.com/catalogue/page-1.html"
while requests.get(new_page).status_code == 200:
    pages_urls.append(new_page)
    new_page = pages_urls[-1].split("-")[0] + "-" + str(int(pages_urls[-1].split("-")[1].split(".")[0]) + 1) + ".html"
    

print(str(len(pages_urls)) + " fetched URLs")
print("Some examples:")
pages_urls[:5]

50 fetched URLs
Some examples:


['http://books.toscrape.com/catalogue/page-1.html',
 'http://books.toscrape.com/catalogue/page-2.html',
 'http://books.toscrape.com/catalogue/page-3.html',
 'http://books.toscrape.com/catalogue/page-4.html',
 'http://books.toscrape.com/catalogue/page-5.html']

In [48]:
booksURLs = []

for page in pages_urls:
    booksURLs.extend(getBooksURLs(page))
    
print(str(len(booksURLs))+ " fetched URLs")
print("some examples:")
booksURLs[:5]
    
    

1000 fetched URLs
some examples:


['http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html',
 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html',
 'http://books.toscrape.com/catalogue/soumission_998/index.html',
 'http://books.toscrape.com/catalogue/sharp-objects_997/index.html',
 'http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html']

In [49]:
names = []
prices = []
nb_in_stock = []
img_urls = []
categories = []
ratings = []

for url in booksURLs:
    soup = getAndParseURL(url)
    
    #product name
    names.append(soup.find("div", class_ = re.compile("product_main")).h1.text)
    
    #product price
    prices.append(soup.find("p", class_ = "price_color").text[2:])
    
    # number of available products
    nb_in_stock.append(re.sub("[^0-9]", "", soup.find("p", class_ = "instock availability").text)) # get rid of non numerical characters
 
    # image url
    img_urls.append(url.replace("index.html", "") + soup.find("img").get("src"))
    
    # product category
    categories.append(soup.find("a", href = re.compile("../category/books/")).get("href").split("/")[3])

    # ratings
    ratings.append(soup.find("p", class_ = re.compile("star-rating")).get("class")[1])

# add data into pandas df
import pandas as pd

scraped_data = pd.DataFrame({'name': names, 'price': prices, 'nb_in_stock': nb_in_stock, "url_img": img_urls, "product_category": categories, "rating": ratings})
scraped_data.head()    

,name,price,nb_in_stock,url_img,product_category,rating
0,A Light in the Attic,51.77,22,http://books.toscrape.com/catalogue/a-light-in...,poetry_23,Three
1,Tipping the Velvet,53.74,20,http://books.toscrape.com/catalogue/tipping-th...,historical-fiction_4,One
2,Soumission,50.10,20,http://books.toscrape.com/catalogue/soumission...,fiction_10,One
3,Sharp Objects,47.82,20,http://books.toscrape.com/catalogue/sharp-obje...,mystery_3,Four
4,Sapiens: A Brief History of Humankind,54.23,20,http://books.toscrape.com/catalogue/sapiens-a-...,history_32,Five


In [ ]:
# Reference Info

# regular expression substitution
# re.sub: https://www.pythonforbeginners.com/regex/regular-expressions-in-python

# string split function        
# split function: https://www.w3schools.com/python/ref_string_split.asp        

# string join function
# https://www.programiz.com/python-programming/methods/string/join